Primera parte del codigo

In [1]:
import re
import time
import json
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.colombiamascompetitiva.com"
START = f"{BASE}/apoyo-a-sectores/"

# --- utilidades --------------------------------------------------------------

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

def norm_text(s: str) -> str:
    if not s:
        return ""
    # colapsa espacios y quita espacios alrededor de puntuación
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\s+([,.;:])", r"\1", s)
    return s.strip()

def get_text(tag) -> str:
    """Extrae texto profundo (incluye spans, strong, etc.)."""
    if tag is None:
        return ""
    return norm_text(tag.get_text(separator=" ", strip=True))

def is_heading(tag) -> bool:
    return tag.name in {"h2", "h3", "h4", "h5"}

def clean_label(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    # normaliza etiquetas conocidas
    mapping = {
        "descripción del proyecto": "descripcion",
        "descripcion del proyecto": "descripcion",
        "factsheet del proyecto": "factsheet",
        "departamentos": "departamentos",
        "municipios": "municipios",
        "ejecutores y aliados": "ejecutores_aliados",
        "duración": "duracion",
        "duracion": "duracion",
        "estado del proyecto": "estado",
        "resultados del proyecto": "resultados",
        "noticias del proyecto": "noticias",
    }
    return mapping.get(s, s)

# --- extracción de enlaces desde el menú -------------------------------------

def collect_menu_links(session: requests.Session, root_url: str, menu_label: str):
    """Obtiene todos los <a> dentro del submenú del ítem cuyo texto visible es menu_label."""
    resp = session.get(root_url, headers=HEADERS, timeout=20)
    soup = BeautifulSoup(resp.text, "html.parser")

    # Matchea por texto exacto, sin sensibilidad a mayúsculas/minúsculas
    menu_anchor = soup.find("a", string=re.compile(rf"^{re.escape(menu_label)}$", re.I))
    if not menu_anchor:
        return []

    parent_li = menu_anchor.find_parent("li")
    if not parent_li:
        return []

    # Tomar solo enlaces dentro de los submenús
    submenus = parent_li.find_all("ul", class_=re.compile(r"\bsub-menu\b"))
    links = []
    for ul in submenus:
        for a in ul.find_all("a", href=True):
            href = a["href"].strip()
            text = get_text(a)
            if href and href != "#" and not href.startswith("mailto:"):
                links.append((text, urljoin(BASE, href)))

    # dedup por URL
    seen = set()
    out = []
    for text, href in links:
        if href not in seen:
            seen.add(href)
            out.append((text, href))
    return out

# --- parseo estructurado de cada página --------------------------------------

def parse_sections(soup: BeautifulSoup):
    """
    Recorre bloques de contenido y construye un dict {seccion: texto}
    También recoge listas y enlaces de cada sección.
    """
    # área típica del contenido en estas páginas WPBakery
    wrappers = soup.select("div.wpb_text_column .wpb_wrapper, div.wpb_wrapper")
    sections = {}
    links_in_page = []

    # fallback si no hay wrappers
    if not wrappers:
        wrappers = [soup.find("main") or soup]

    current_key = None
    bucket = []

    def flush():
        nonlocal current_key, bucket
        if current_key:
            text = norm_text(" ".join(bucket))
            if text:
                # acumula si la sección aparece varias veces
                sections[current_key] = (sections.get(current_key, "") + " " + text).strip()
        bucket = []

    for wrapper in wrappers:
        # recorre hijos directos para respetar el orden
        for child in wrapper.children:
            if getattr(child, "name", None) is None:
                continue

            if is_heading(child):
                # volcar lo anterior y empezar una nueva sección
                flush()
                current_key = clean_label(get_text(child))
                continue

            # agrega contenido textual (p, div, span conteniendo texto)
            if child.name in {"p", "div", "span"}:
                txt = get_text(child)
                if txt:
                    bucket.append(txt)

            # listas
            if child.name in {"ul", "ol"}:
                items = [get_text(li) for li in child.find_all("li")]
                items = [it for it in items if it]
                if items:
                    bucket.append(" | ".join(items))

            # enlaces que puedan ser útiles (factsheets, noticias, etc.)
            for a in child.find_all("a", href=True):
                href = urljoin(BASE, a["href"])
                txt = get_text(a)
                if href and href != "#" and not href.startswith("mailto:"):
                    links_in_page.append({"text": txt, "href": href})

    # volcar último bucket
    flush()

    return sections, links_in_page

def scrape_detail(session: requests.Session, url: str):
    r = session.get(url, headers=HEADERS, timeout=30)
    s = BeautifulSoup(r.text, "html.parser")
    h1 = get_text(s.find("h1")) or "Sin título"
    sections, links_in_page = parse_sections(s)

    # texto plano completo (útil para búsquedas / filtros)
    all_text = " ".join([h1] + list(sections.values()))
    all_text = norm_text(all_text)

    # campos comunes si existen
    row = {
        "url": url,
        "title": h1,
        "descripcion": sections.get("descripcion", ""),
        "factsheet": sections.get("factsheet", ""),
        "departamentos": sections.get("departamentos", ""),
        "municipios": sections.get("municipios", ""),
        "ejecutores_aliados": sections.get("ejecutores_aliados", ""),
        "duracion": sections.get("duracion", ""),
        "estado": sections.get("estado", ""),
        "resultados": sections.get("resultados", ""),
        "noticias": sections.get("noticias", ""),
        "sections_json": json.dumps(sections, ensure_ascii=False),
        "links_json": json.dumps(links_in_page, ensure_ascii=False),
        "raw_text": all_text,
    }
    return row

# --- ejecución ---------------------------------------------------------------

session = requests.Session()

# Enlaces finales de ambos menús
links_cadenas = collect_menu_links(session, START, "Cadenas de Valor")
links_convoc = collect_menu_links(session, START, "Convocatorias")

menu_links = links_cadenas + links_convoc
print(f"Total enlaces del menú: {len(menu_links)}")

# Scrape detallado
rows = []
seen_urls = set()
for label, link in menu_links:
    if link in seen_urls:
        continue
    seen_urls.add(link)
    try:
        print("→", link)
        rows.append(scrape_detail(session, link))
        time.sleep(0.6)  # educación con el servidor
    except Exception as e:
        print(f"[ERROR] {link}: {e}")

df = pd.DataFrame(rows)

# Guarda un CSV “rico” para luego filtrar por departamentos/municipios
out = "cadenas_convocatorias_detallado.csv"
df.to_csv(out, index=False, encoding="utf-8-sig")
print(f"\n✅ Archivo generado: {out}")
print(df[["title", "url"]].head(8))


Total enlaces del menú: 69
→ https://www.colombiamascompetitiva.com/cadena-de-valor-turismo-sostenible/
→ https://www.colombiamascompetitiva.com/destino-llanos/
→ https://www.colombiamascompetitiva.com/destino-putumayo/
→ https://www.colombiamascompetitiva.com/macondo-natural/
→ https://www.colombiamascompetitiva.com/trekking-en-paisaje-cultural-cafetero/
→ https://www.colombiamascompetitiva.com/cadena-de-valor-turismo-sostenible/una-aventura-magica/
→ https://www.colombiamascompetitiva.com/cadena-de-valor-turismo-sostenible/san-agustin-turismo-competitivo/
→ https://www.colombiamascompetitiva.com/cadena-de-valor-turismo-sostenible/macondo-natural/
→ https://www.colombiamascompetitiva.com/cadena-de-valor-turismo-sostenible/corredor-cordillerano/
→ https://www.colombiamascompetitiva.com/iniciativa-destinos-competitivos-sostenibles-2/
→ https://www.colombiamascompetitiva.com/dcsii/
→ https://www.colombiamascompetitiva.com/reestructuracion-fontur/
→ https://www.colombiamascompetitiva.com/

Segunda parte del codigo

In [2]:
import pandas as pd

# Cargar el archivo detallado
df = pd.read_csv("cadenas_convocatorias_detallado.csv")

# --------------------------
# Diccionario oficial de municipios
# --------------------------
geo_filter = {
    "Huila": [
        "Neiva","Acevedo","Agrado","Aipe","Algeciras","Altamira","Baraya","Campoalegre","Colombia","Elías",
        "Garzón","Gigante","Guadalupe","Hobo","Íquira","Isnos","La Argentina","La Plata","Nátaga","Oporapa",
        "Paicol","Palermo","Palestina","Pital","Pitalito","Rivera","Saladoblanco","San Agustín",
        "Santa María","Suaza","Tarqui","Tello","Teruel","Tesalia","Timaná","Villavieja","Yaguará"
    ],
    "Tolima": [
        "Ibagué","Alpujarra","Alvarado","Ambalema","Anzoátegui","Armero","Ataco","Cajamarca","Carmen de Apicalá",
        "Casabianca","Chaparral","Coello","Coyaima","Cunday","Dolores","Espinal","Falan","Flandes","Fresno",
        "Guamo","Herveo","Honda","Icononzo","Lérida","Líbano","Mariquita","Melgar","Murillo","Natagaima",
        "Ortega","Palocabildo","Piedras","Planadas","Prado","Purificación","Rioblanco","Roncesvalles",
        "Rovira","Saldaña","San Antonio","San Luis","Santa Isabel","Suárez","Valle de San Juan",
        "Venadillo","Villahermosa","Villarrica"
    ],
    "Putumayo": [
        "Mocoa","Colón","Orito","Puerto Asís","Puerto Caicedo","Puerto Guzmán","Puerto Leguízamo",
        "San Francisco","San Miguel","Santiago","Sibundoy","Valle del Guamuez","Villagarzón"
    ],
    "Caquetá": [
        "Florencia","Albania","Belén de los Andaquíes","Cartagena del Chairá","Curillo","El Doncello",
        "El Paujíl","La Montañita","Milán","Morelia","Puerto Rico","San José del Fragua",
        "San Vicente del Caguán","Solano","Solita","Valparaíso"
    ]
}

# --------------------------
# Construir lista de keywords
# --------------------------
keywords = [dep.lower() for dep in geo_filter.keys()]
for munis in geo_filter.values():
    keywords.extend([m.lower() for m in munis])

# --------------------------
# Filtrar por coincidencia en cualquier columna de texto
# --------------------------
def contiene_palabra(texto, palabras):
    texto = str(texto).lower()
    return any(p in texto for p in palabras)

mask = df.apply(
    lambda row: contiene_palabra(row["raw_text"], keywords),
    axis=1
)

df_filtrado = df[mask]

# --------------------------
# Guardar resultados
# --------------------------
df_filtrado.to_csv("proyectos_filtrados.csv", index=False, encoding="utf-8-sig")
print("✅ Archivo filtrado guardado como proyectos_filtrados.csv")
print(df_filtrado[["title", "departamentos", "municipios"]])


✅ Archivo filtrado guardado como proyectos_filtrados.csv
                                                title  departamentos  \
2                                    DESTINO PUTUMAYO            NaN   
6                                         San Agustín            NaN   
10              Destinos + Competitivos + Sostenibles            NaN   
11                            Reestructuración FONTUR            NaN   
14                                     NUEVA REALIDAD            NaN   
15                                         Sin título            NaN   
16  Transformando el Futuro de los Ingredientes Na...            NaN   
18  Cultivando el Futuro: Producción Sostenible y ...            NaN   
19                 Ecosistemas que Impulsan el Cambio            NaN   
20                Herramientas para la Competitividad            NaN   
26              Frutos de la Biodiversidad Colombiana            NaN   
28                                     PLAN DE ACCIÓN            NaN   
29   In